In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import statistics
from current_model import *
import datetime

In [18]:
from sklearn.metrics import mean_squared_error

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

## Function to get the real value and the predicted amount of backwater caused by vegetation

In [19]:
def analyze_pred_veg2(dat,weir,last_days,avg_temp,risk_date):
    data = dat[dat['STUWVAK'] == weir]
    data.reset_index(inplace=True, drop=True)
    real_vegetation = (data[data['index'] > risk_date].head(21)).reset_index()
    data = data[data['index'] <= risk_date]
    last_data=data.tail(last_days)
    last_data.rename(columns = {"index": "TIME"}, inplace = True)
    last_day = datetime.datetime.strptime(last_data.iloc[-1]['TIME'], "%Y-%m-%d")
    new_dates=[last_day+datetime.timedelta(days=i) for i in range(1,22)]
    last_data.reset_index(inplace=True)
    # Define linear model
    reg = linear_model.LinearRegression()
    # Take index and the back water by vegetation as training data
    x_train=last_data.index.to_numpy().reshape(-1, 1)
    y_train=last_data['Real backwater caused by vegetation'].to_numpy().reshape(-1, 1)
    # Fit the linear model on the last days
    reg.fit(x_train,y_train)
    # Get index for the next 21 days
    x_test=[x_train[-1]+i for i in range(1,22)]
    # Predict the vegetation for the next 21 days
    predictions=reg.predict(x_test)
    # Format
    predictions=[item for elem in predictions.tolist() for item in elem]
    # Depending on the temperature add multplication value to adjust values
    try:
        if (avg_temp > 25):
            predictions=[pred*1.2 for pred in predictions]
        elif (avg_temp<20):
            predictions=[pred*0.8 for pred in predictions]
    except:
        print("The Temperature was not available")
    data = {'TIME':  new_dates,'Predicted backwater by vegetation': predictions}
    df = pd.DataFrame(data, columns = ['TIME','Predicted backwater by vegetation'])
    df['Real backwater'] = real_vegetation['Real backwater caused by vegetation'] ## CHANGED VERSCHIL TO 'Real backwater caused by vegetation'
    return df

## Function to get the mae

In [20]:
data_path = 'feature_tables/feature_tables/'

In [21]:
from sklearn.metrics import mean_absolute_error

def get_mae(dat,weir,last_days,avg_temp):#(weir,data_path,last_days,avg_temp):
    #data=get_data(weir,data_path,date_format=True).reset_index()
    data = dat[dat['STUWVAK'] == weir]
    lst = []
    lst1 = []
    lst_dates = data['index'].iloc[6:len(data)-21].to_list()[0::21] # Select 1 in every 10 dates in the data set#data['index'].iloc[6:len(data)-21].to_list()
    #print(len(lst_dates))
    #i=1
    for risk_date in lst_dates:
        #print(i)
        df = analyze_pred_veg2(data,weir,last_days,avg_temp,risk_date)
        #mse = mean_squared_error(df['Real backwater'],df['Predicted backwater by vegetation'])
        mae = mean_absolute_error(df['Real backwater'],df['Predicted backwater by vegetation'])
        lst.append(mae)
        lst1.append(df['Predicted backwater by vegetation'].mean())
        #i+=1
    
    df_mae = pd.DataFrame(index=lst_dates)
    df_mae['mae'] = lst
    df_mae['average predicted value'] = lst1
    df_mae['Real Backwater'] = lst2
    print(df_mae)
    return df_mae

## Calculate Real Backwater

In [22]:
data = pd.read_csv('All_weirs.csv', parse_dates=True)

In [23]:
def det_station(x):
    if x[5:7] == '01' or x[5:7] == '02':
        return x[:4] + 'Winter'
    elif x[5:7] == '10' or x[5:7] == '11' or x[5:7] == '12':
        return str(int(x[:4]) + 1) + 'Winter'
    else: 
        return x[:4] + 'Summer'
    

In [24]:
data['month_year'] = data['TIME'].apply(lambda x : det_station(x))

In [25]:
median_winter_weirs = {}
for weir in data['STUWVAK'].unique():
    weir_data = data[data['STUWVAK'] == weir]
    statdict = {}
    for station in weir_data['month_year'].unique():
        if station[4:] == 'Winter':
            stat = weir_data[weir_data['month_year'] == station]
            if len(stat) > 0:
                statdict[station] = stat['VERSCHIL'].median()
    median_winter_weirs[weir] = statdict

In [27]:
data['Real Backwater caused by Vegetation'] = 0.0
i = 0
while i < len(data):
    if data['month_year'][i][4:] == 'Summer':
        if data['month_year'][i][:4] + 'Winter' in median_winter_weirs[data['STUWVAK'][i]].keys():
            data['Real Backwater caused by Vegetation'][i] = data['VERSCHIL'][i] - median_winter_weirs[data['STUWVAK'][i]][data['month_year'][i][:4] + 'Winter']
        else:
            data['Real Backwater caused by Vegetation'][i] = 'No winter data for the selected data'
    i += 1


## Start analyzing the assigned data

In [36]:
awbv = pd.read_csv('Allweirs with backwater caused by vegetation.csv')
awbv = data
#awbv[awbv['STUWVAK'] == '211B_211A'].reset_index(drop=True)

lsttt = [str(datetime.datetime.strptime(dt, '%Y-%m-%dT%H:%M:%SZ').date()) for dt in awbv['TIME']]
awbv['index'] = lsttt
awbv.drop(columns=['TIME'], inplace=True)
awbv.rename(columns={'Real Backwater caused by Vegetation': 'Real backwater caused by vegetation'}, inplace=True)
awbv = awbv[awbv['Real backwater caused by vegetation'] != 'No winter data for the selected data'].reset_index(drop=True)
awbv

,STUWVAK,VERSCHIL,Q,month_year,Real backwater caused by vegetation,index
0,103BIC_103BIB,0.082615,0.075021,2021Winter,0.0,2020-10-01
1,103BIC_103BIB,0.080000,0.019067,2021Winter,0.0,2020-10-02
2,103BIC_103BIB,0.093250,0.003626,2021Winter,0.0,2020-10-03
3,103BIC_103BIB,0.087139,0.001093,2021Winter,0.0,2020-10-04
4,103BIC_103BIB,0.087000,0.000380,2021Winter,0.0,2020-10-05
...,...,...,...,...,...,...
34717,251U_251T,0.141594,0.395848,2021Summer,-0.029083,2021-07-12
34718,251U_251T,0.136792,0.400390,2021Summer,-0.033886,2021-07-13
34719,251U_251T,0.137073,0.398823,2021Summer,-0.033604,2021-07-14
34720,251U_251T,0.148073,0.401904,2021Summer,-0.022604,2021-07-15


In [ ]:
df_cv = pd.DataFrame()
df_cv['Last Days Parameter'] = list(range(2, 31))
df_cv['MAE'] = 0.0
for nr in range(2, 31): 
    maes_all = []
    for weir in awbv['STUWVAK'].unique():
        weir_data = awbv[awbv['STUWVAK'] == weir].reset_index()
        weir_data = weir_data.drop(columns = ['level_0'])
        weir_data['Predicted Backwater by Vegetation'] = 0.0
        i = nr
        while i < len(weir_data):
            risk_date = weir_data['index'][i]
            days21pred = analyze_pred_veg2(weir_data, weir, nr,avg_temp,risk_date)
            n = 0
            while n < len(days21pred):
                if i + n < len(weir_data):
                    weir_data['Predicted Backwater by Vegetation'][i + n] = days21pred['Predicted backwater by vegetation'][n]
                n += 1
            i += 21
        weir_maes = []
        for year in weir_data['month_year'].unique():
            if year[4:] == 'Summer':
                yeardata = weir_data[weir_data['month_year'] == year]
                c = 0
                while c < len(yeardata):
                    if weir_data['Predicted Backwater by Vegetation'][c] - weir_data['Real backwater caused by vegetation'][c] > 0:
                        weir_maes.append(weir_data['Predicted Backwater by Vegetation'][c] - weir_data['Real backwater caused by vegetation'][c])
                    else:
                        weir_maes.append(weir_data['Real backwater caused by vegetation'][c] - weir_data['Predicted Backwater by Vegetation'][c])
                    c += 1
        maes_all.append(np.mean(weir_maes))
    df_cv['MAE'][nr - 2] = np.mean(maes_all)
                
                        
                        
                

In [ ]:
plt.plot(df_cv['MAE'], df_cv['Nr of Last Days'])
plt.ylabel('MAE')
plt.xlabel('Last Days Parameter')
plt.title('MAE depending on the different values of the last day parameter')

In [ ]:
last_days = 18
alldfs = []
for weir in Jorgeweirs['STUWVAK'].unique():
    weir_data = Jorgeweirs[Jorgeweirs['STUWVAK'] == weir].reset_index()
    weir_data['Predicted Backwater by Vegetation'] = 0.0
    i = 0
    while i < len(weir_data):
        risk_date = weir_data['index'][i]
        days21pred = analyze_pred_veg2(Jorgeweirs, weir, last_days,avg_temp,risk_date)
        n = 0
        while n < len(days21pred):
            if i + n < len(weir_data):
                weir_data['Predicted Backwater by Vegetation'][i + n] = days21pred['Predicted backwater by vegetation'][n]
            n += 1
        i += 21
    alldfs.append(weir_data)

In [ ]:
import math
maes_dict = {}
for weir in alldfs:
    weir['year'] = weir['month_year'].apply(lambda x : x[:4])
    weir = weir.drop(columns = ['level_0'])
    all_years = {}
    for year in weir['year'].unique():
        df_year = weir[weir['month_year'] == year + 'Summer']
        if len(df_year) > 1:
            df_year['Real backwater caused by vegetation'] = df_year['Real backwater caused by vegetation'].apply(lambda x : float(x))
            df_year = df_year.reset_index()
            i = 0 
            list_maes = []
            while i < len(df_year):
                if df_year['Predicted Backwater by Vegetation'][i] - df_year['Real backwater caused by vegetation'][i] < 0:
                    list_maes.append(df_year['Real backwater caused by vegetation'][i] - df_year['Predicted Backwater by Vegetation'][i])
                else: 
                    list_maes.append(df_year['Predicted Backwater by Vegetation'][i] - df_year['Real backwater caused by vegetation'][i])
                i += 1
            all_years[df_year['year'].unique()[0]] = list_maes
    maes_dict[weir['STUWVAK'][0]] = all_years

In [ ]:

stds_dict = {}
for weir in alldfs:
    weir['year'] = weir['month_year'].apply(lambda x : x[:4])
    weir = weir.drop(columns = ['level_0'])
    all_years = {}
    for year in weir['year'].unique():
        df_year = weir[weir['month_year'] == year + 'Summer']
        if len(df_year) > 1:
            df_year['Real backwater caused by vegetation'] = df_year['Real backwater caused by vegetation'].apply(lambda x : float(x))
            variance = df_year['Real backwater caused by vegetation'].var()
            std = math.sqrt(variance)
            df_year = df_year.reset_index()
            i = 0 
            list_stds = []
            while i < len(df_year):
                if df_year['Predicted Backwater by Vegetation'][i] - df_year['Real backwater caused by vegetation'][i] < 0:
                    list_stds.append((df_year['Real backwater caused by vegetation'][i] - df_year['Predicted Backwater by Vegetation'][i]) / std)
                else: 
                    list_stds.append((df_year['Predicted Backwater by Vegetation'][i] - df_year['Real backwater caused by vegetation'][i]) / std)
                i += 1
            all_years[df_year['year'].unique()[0]] = list_stds
    stds_dict[weir['STUWVAK'][0]] = all_years

In [ ]:
columns = ['STUWVAK', 'MAE']
stuwvak = [''] * 20000
mae = [0.0] * 20000
data2018 = pd.DataFrame(columns = columns)
data2018['MAE'] = mae
data2018['STD'] = mae
data2018['STUWVAK'] = stuwvak
data2019 = pd.DataFrame(columns = columns)
data2019['MAE'] = mae
data2019['STD'] = mae
data2019['STUWVAK'] = stuwvak
data2020 = pd.DataFrame(columns = columns)
data2020['MAE'] = mae
data2020['STD'] = mae
data2020['STUWVAK'] = stuwvak
data2021 = pd.DataFrame(columns = columns)
data2021['MAE'] = mae
data2021['STD'] = mae
data2021['STUWVAK'] = stuwvak
i = 0
x = 0
y = 0
z = 0
for weir in maes_dict.keys():
    if '2018' in maes_dict[weir].keys():
        for mae in maes_dict[weir]['2018']:
            data2018['STUWVAK'][i] = weir 
            data2018['MAE'][i] = mae
            i += 1
    if '2019' in maes_dict[weir].keys():
        for mae in maes_dict[weir]['2019']:
            data2019['STUWVAK'][x] = weir
            data2019['MAE'][x] = mae
            x += 1
    if '2020' in maes_dict[weir].keys():
        for mae in maes_dict[weir]['2020']:
            data2020['STUWVAK'][y] = weir 
            data2020['MAE'][y] = mae
            y += 1
    if '2021' in maes_dict[weir].keys():
        for mae in maes_dict[weir]['2021']:
            data2021['STUWVAK'][z] = weir
            data2021['MAE'][z] = mae
            z += 1
i = 0
x = 0
y = 0
z = 0
for weir in stds_dict.keys():
    if '2018' in stds_dict[weir].keys():
        for std in stds_dict[weir]['2018']:
            data2018['STUWVAK'][i] = weir 
            data2018['STD'][i] = std
            i += 1
    if '2019' in stds_dict[weir].keys():
        for std in stds_dict[weir]['2019']:
            data2019['STUWVAK'][x] = weir
            data2019['STD'][x] = std
            x += 1
    if '2020' in stds_dict[weir].keys():
        for std in maes_dict[weir]['2020']:
            data2020['STUWVAK'][y] = weir 
            data2020['STD'][y] = std
            y += 1
    if '2021' in stds_dict[weir].keys():
        for std in stds_dict[weir]['2021']:
            data2021['STUWVAK'][z] = weir
            data2021['STD'][z] = std
            z += 1

In [ ]:
data2018 = data2018[data2018['STUWVAK'] != '']
data2019 = data2019[data2019['STUWVAK'] != '']
data2020 = data2020[data2020['STUWVAK'] != '']
data2021 = data2021[data2021['STUWVAK'] != '']

In [ ]:
data2021short = data2021[data2021['STUWVAK'].isin(data2021['STUWVAK'].unique()[:19])]

In [ ]:
for df in alldfs:
    df['Change from yesterday'] = 0.0
    i = 0
    df['Real backwater caused by vegetation'] = df['Real backwater caused by vegetation'].apply(lambda x : float(x))
    while i < len(df):
        if i >= 1:
            if df['Real backwater caused by vegetation'][i] - df['Real backwater caused by vegetation'][i - 1] > 0:
                df['Change from yesterday'][i] = df['Real backwater caused by vegetation'][i] - df['Real backwater caused by vegetation'][i - 1]
            else:
                df['Change from yesterday'][i] = df['Real backwater caused by vegetation'][i - 1] - df['Real backwater caused by vegetation'][i]
        i += 1

In [ ]:
import seaborn as sns
plt.figure(figsize=(30,20))
plt.title('Absolute Errors for each weir in 2021', size = 30)
sns.boxplot(x='STUWVAK', y='MAE', data= data2021)
plt.ylim(top=1)
plt.ylabel('Absolute Error', size = 20)
plt.xlabel('STUWVAK', size = 20)
plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize=(30,20))
plt.title('NR of STDS of AE for each weir in 2021', size = 30)
sns.boxplot(x='STUWVAK', y='STD', data= data2021)
plt.ylim(top=8)
plt.ylabel('STDS', size = 20)
plt.xlabel('STUWVAK', size = 20)
plt.show()

In [ ]:
bestweirs = data2021.groupby('STUWVAK')['MAE'].median().to_frame().reset_index().sort_values(by=['MAE'],ascending=True)[:10]['STUWVAK'].to_list()
worstweirs = data2021.groupby('STUWVAK')['MAE'].median().to_frame().reset_index().sort_values(by=['MAE'],ascending=False)

In [ ]:
data2021.groupby('STUWVAK')['MAE'].median().to_frame().reset_index().sort_values(by=['MAE'],ascending=False)

In [ ]:
data2021.groupby('STUWVAK')['MAE'].median().to_frame().reset_index().sort_values(by=['MAE'],ascending=False)

In [ ]:
best_worst = ['211C_211B', '211L_211K']

In [ ]:
bestweirs = data2021.groupby('STUWVAK')['STD'].median().to_frame().reset_index().sort_values(by=['STD'],ascending=True)[:10]['STUWVAK'].to_list()
worstweirs = data2021.groupby('STUWVAK')['STD'].median().to_frame().reset_index().sort_values(by=['STD'],ascending=False)[:10]['STUWVAK'].to_list()

In [ ]:
dataresults = data2021.groupby('STUWVAK')['MAE'].median().to_frame().reset_index().sort_values(by=['MAE'],ascending=True)

In [ ]:
dataresults['Change from one day to another'] = 0.0
i = 0
for stuwvak in dataresults['STUWVAK']:
    for df in alldfs:
        if stuwvak == df['STUWVAK'][0]:
            df = df[df['month_year'] == '2021Summer']
            df['Real backwater caused by vegetation'] = df['Real backwater caused by vegetation'].apply(lambda x : float(x))
            rangev = df['Change from yesterday'].mean()
            dataresults['Change from one day to another'][i] = rangev
    i += 1

dataresults['Range'] = 0.0
i = 0
for stuwvak in dataresults['STUWVAK']:
    for df in alldfs:
        if stuwvak == df['STUWVAK'][0]:
            df = df[df['month_year'] == '2021Summer']
            df['Real backwater caused by vegetation'] = df['Real backwater caused by vegetation'].apply(lambda x : float(x))
            rangev = df['Real backwater caused by vegetation'].max() - df['Real backwater caused by vegetation'].min()
            dataresults['Range'][i] = rangev
    i += 1
dataresults['Mean'] = 0.0
i = 0
for stuwvak in dataresults['STUWVAK']:
    for df in alldfs:
        if stuwvak == df['STUWVAK'][0]:
            df = df[df['month_year'] == '2021Summer']
            df['Real backwater caused by vegetation'] = df['Real backwater caused by vegetation'].apply(lambda x : float(x))
            rangev = df['Real backwater caused by vegetation'].mean()
            dataresults['Mean'][i] = rangev
    i += 1

dataresults['Variation'] = 0.0
i = 0
for stuwvak in dataresults['STUWVAK']:
    for df in alldfs:
        if stuwvak == df['STUWVAK'][0]:
            df = df[df['month_year'] == '2021Summer']
            df['Real backwater caused by vegetation'] = df['Real backwater caused by vegetation'].apply(lambda x : float(x))
            rangev = df['Real backwater caused by vegetation'].var()
            dataresults['Variation'][i] = rangev
    i += 1

In [ ]:
dataresults

In [ ]:
import seaborn as sns
sns.regplot(x=dataresults['MAE'], y=dataresults["Change from one day to another"])
plt.ylabel('Average Change of Backwater')
plt.xlabel('Mean Absolute Error')
plt.title('Mean Absolute Error depending on the average change of backwater from one day to another of a stuwvak')

In [ ]:
dataresults = dataresults[dataresults['Range'] < 1.25]